In [12]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, explode, split, array_distinct, udf
# from pyspark.ml.feature import MinHashLSH
# from pyspark.sql.types import FloatType
# import pyspark.sql.functions as F
# import os
# import numpy as np
# import gcsfs
# from pyspark.ml.linalg import Vectors, VectorUDT
# from pyspark.sql.functions import udf
# from itertools import combinations

# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, split, explode, collect_set, lit, array, udf
# from pyspark.sql.types import DoubleType
# from itertools import combinations

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, split, collect_list, flatten, 
    array_intersect, array_union, size,
    when, first
)
from pyspark.sql.functions import lit
from pyspark.storagelevel import StorageLevel

In [13]:
spark = SparkSession.builder.getOrCreate()

# Similarity matrix for propagation results

For future speed optiomisation use Minhash for similarity calculation.

## Jaccard

In [14]:
def calculate_jaccard_similarity(base_gcs_path, folders_to_process, output_gcs_path):
    spark.conf.set("spark.sql.shuffle.partitions", "200")
    
    for folder in folders_to_process:
        input_path = f"{base_gcs_path}/{folder}/**/*.parquet"
        df = spark.read.option("recursiveFileLookup", "true").parquet(input_path)
        
        # Process terms more efficiently
        df = df.withColumn("term", split(col("term"), ","))
        
        grouped = df.groupBy("approvedSymbol").agg(
            flatten(collect_list("term")).alias("terms"),
            first("taregtId").alias("taregtId")
        ).cache()
        
        # Self-join with filtering
        pairs = grouped.alias("a").join(
            grouped.alias("b"),
            col("a.approvedSymbol") < col("b.approvedSymbol")
        ).filter(
            size(array_intersect(col("a.terms"), col("b.terms"))) > 0
        )
        
        # Calculate Jaccard without UDF
        result = pairs.withColumn(
            "jaccardSimilarity",
            size(array_intersect(col("a.terms"), col("b.terms"))) / 
            size(array_union(col("a.terms"), col("b.terms")))
        ).select(
            col("a.approvedSymbol").alias("approvedSymbolA"),
            col("b.approvedSymbol").alias("approvedSymbolB"),
            col("a.taregtId").alias("taregtIdA"),
            col("b.taregtId").alias("taregtIdB"),
            col("jaccardSimilarity")
        )
        
        output_path = f"{output_gcs_path}/{folder}/jaccard_output"
        result.write.mode("overwrite").parquet(output_path)
        grouped.unpersist()

In [ ]:
gsea_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark"
output_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/similarity_mtx/jaccard_spark"

calculate_jaccard_similarity(
    base_gcs_path=gsea_dir,
    folders_to_process=["test"],
    output_gcs_path=output_dir
)

25/05/01 11:09:13 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [57]:
df_show = spark.read.csv("gs://ot-team/polina/pathway_propagation_validation_v2/similarity_mtx/jaccard_spark/KEGG_2021_Human/EFO_0000565_ge_mm_som_gsea_KEGG_2021_Human_pval0.05", header=True, inferSchema=True)

In [62]:
df_show.filter(col("targetA") == "CDK2").show(20)
# df_show.show(10)

+-------+----------+-----------+
|targetA|   targetB| jaccardSim|
+-------+----------+-----------+
|   CDK2|      CDK2|        1.0|
|   CDK2|      CDK4| 0.41935483|
|   CDK2|      CDK5|        0.0|
|   CDK2|    CDK5R1|        0.0|
|   CDK2|      CDK6| 0.41379312|
|   CDK2|      CDK7|0.055555556|
|   CDK2|      CDK9|        0.0|
|   CDK2|    CDKN1A|  0.3902439|
|   CDK2|    CDKN1B|        0.5|
|   CDK2|    CDKN1C|0.055555556|
|   CDK2|    CDKN2A| 0.25925925|
|   CDK2|    CDKN2B|        0.5|
|   CDK2|    CDKN2C| 0.15789473|
|   CDK2|    CDKN2D| 0.11111111|
|   CDK2|      CDX2|0.055555556|
|   CDK2|     CEBPA|       0.05|
|   CDK2|     CEBPB|        0.0|
|   CDK2|     CEBPE|        0.0|
|   CDK2|     CENPS|        0.0|
|   CDK2|CENPS-CORT|        0.0|
+-------+----------+-----------+
only showing top 20 rows



Ok it's too long. Lets return to hash-tables.